You can use this exercise as a guide to predict house prices using numerical in a real dataset. It is based on this tutorial https://www.pyimagesearch.com/2019/01/28/keras-regression-and-cnns/.

In [ ]:
# Downloading the data
!git clone https://github.com/emanhamed/Houses-dataset
!ls -la Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (20/20), done.
total 96
drwxr-xr-x 4 root root  4096 May 24 20:56  .
drwxr-xr-x 1 root root  4096 May 24 20:56  ..
drwxr-xr-x 8 root root  4096 May 24 20:56  .git
-rw-r--r-- 1 root root   378 May 24 20:56  .gitattributes
-rw-r--r-- 1 root root  1726 May 24 20:56  .gitignore
drwxr-xr-x 2 root root 73728 May 24 20:56 'Houses Dataset'
-rw-r--r-- 1 root root  1767 May 24 20:56  README.md


In [ ]:
!ls -la Houses-dataset/Houses\ Dataset

total 186552
drwxr-xr-x 2 root root  73728 May 24 20:56 .
drwxr-xr-x 4 root root   4096 May 24 20:56 ..
-rw-r--r-- 1 root root 428033 May 24 20:56 100_bathroom.jpg
-rw-r--r-- 1 root root  42411 May 24 20:56 100_bedroom.jpg
-rw-r--r-- 1 root root 282545 May 24 20:56 100_frontal.jpg
-rw-r--r-- 1 root root 223147 May 24 20:56 100_kitchen.jpg
-rw-r--r-- 1 root root 259472 May 24 20:56 101_bathroom.jpg
-rw-r--r-- 1 root root 276635 May 24 20:56 101_bedroom.jpg
-rw-r--r-- 1 root root 234326 May 24 20:56 101_frontal.jpg
-rw-r--r-- 1 root root 315347 May 24 20:56 101_kitchen.jpg
-rw-r--r-- 1 root root 307371 May 24 20:56 102_bathroom.jpg
-rw-r--r-- 1 root root 263332 May 24 20:56 102_bedroom.jpg
-rw-r--r-- 1 root root 291815 May 24 20:56 102_frontal.jpg
-rw-r--r-- 1 root root 234550 May 24 20:56 102_kitchen.jpg
-rw-r--r-- 1 root root 184084 May 24 20:56 103_bathroom.jpg
-rw-r--r-- 1 root root 161703 May 24 20:56 103_bedroom.jpg
-rw-r--r-- 1 root root 424171 May 24 20:56 103_frontal.jpg
-rw-r--

We'll create images such these:

<img src="https://www.pyimagesearch.com/wp-content/uploads/2019/01/keras_regression_cnns_houses.jpg">

Each instance wil be located in the same position. This is key to train the model. We should be consistent!

In [ ]:
# import the necessary packages
import pandas as pd
import numpy as np
import glob
import cv2
import os

from tqdm import tqdm

def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
 
    # determine (1) the unique zip codes and (2) the number of data
	# points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()
	# loop over each of the unique zip codes and their corresponding
	# count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)
	# return the data frame
	return df


def load_house_images(df, inputPath):
	# initialize our images array (i.e., the house images themselves)
	images = []
	# loop over the indexes of the houses
	for i in tqdm(df.index.values):
		# find the four images for the house and sort the file paths,
		# ensuring the four are always in the *same order*
		basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
		housePaths = sorted(list(glob.glob(basePath)))
  
  		# initialize our list of input images along with the output image
		# after *combining* the four input images
		inputImages = []
		outputImage = np.zeros((64, 64, 3), dtype="uint8")
		# loop over the input house paths
		for housePath in housePaths:
			# load the input image, resize it to be 32 32, and then
			# update the list of input images
			image = cv2.imread(housePath)
			image = cv2.resize(image, (32, 32))
			inputImages.append(image)
   
   	# tile the four input images in the output image such the first
		# image goes in the top-right corner, the second image in the
		# top-left corner, the third image in the bottom-right corner,
		# and the final image in the bottom-left corner
		outputImage[0:32, 0:32] = inputImages[0]
		outputImage[0:32, 32:64] = inputImages[1]
		outputImage[32:64, 32:64] = inputImages[2]
		outputImage[32:64, 0:32] = inputImages[3]
		# add the tiled image to our set of images the network will be trained on
		images.append(outputImage)
	# return our set of images
	return np.array(images)


Let's define the model as in the tutorial:

<img src="https://pyimagesearch.com/wp-content/uploads/2019/01/keras_regression_cnns_flow.jpg" style="width:500px;height:750px;">

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input
	inputs = Input(shape=inputShape)
	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input appropriately
		if i == 0:
			x = inputs
		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Activation("relu")(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
	
	# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Activation("relu")(x)
	x = Dropout(0.5)(x)
	# apply another FC layer, this one to match the number of nodes coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)
	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)
	# construct the CNN
	model = Model(inputs, x)
	# return the CNN
	return model

In [ ]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import locale
import os

# construct the path to the input .txt file that contains information on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
# inputPath = os.path.sep.join([args["dataset"], "HousesInfo.txt"])
inputPath = "Houses-dataset/Houses Dataset/HousesInfo.txt"
df = load_house_attributes(inputPath)

# load the house images and then scale the pixel intensities to the range [0, 1]
print("[INFO] loading house images...")
images = load_house_images(df, "Houses-dataset/Houses Dataset/")
images = images / 255.0

# partition the data into training and testing splits using 75% of the data for training and the remaining 25% for testing
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

  1%|          | 2/362 [00:00<00:19, 18.50it/s]

[INFO] loading house attributes...
[INFO] loading house images...


100%|██████████| 362/362 [00:11<00:00, 30.93it/s]


In [ ]:
print(images.shape)
print(trainImagesX.shape)

(362, 64, 64, 3)
(271, 64, 64, 3)


In [ ]:
# find the largest house price in the training set and use it to scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [ ]:
# create our Convolutional Neural Network and then compile the model using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our price *predictions* and the *actual prices*
model = create_cnn(64, 64, 3, regress=True)

opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

# train the model
print("[INFO] training model...")
model.fit(x=trainImagesX, y=trainY, 
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)

[INFO] training model...
Epoch 1/200
34/34 [==============================] - 3s 64ms/step - loss: 1038.4724 - val_loss: 78.3642
Epoch 2/200
34/34 [==============================] - 2s 50ms/step - loss: 727.3735 - val_loss: 74.4343
Epoch 3/200
34/34 [==============================] - 2s 51ms/step - loss: 850.1568 - val_loss: 74.5841
Epoch 4/200
34/34 [==============================] - 2s 51ms/step - loss: 363.5221 - val_loss: 72.0189
Epoch 5/200
34/34 [==============================] - 2s 50ms/step - loss: 324.4216 - val_loss: 71.6350
Epoch 6/200
34/34 [==============================] - 2s 49ms/step - loss: 223.0686 - val_loss: 69.8721
Epoch 7/200
34/34 [==============================] - 2s 50ms/step - loss: 236.2274 - val_loss: 71.4938
Epoch 8/200
34/34 [==============================] - 2s 52ms/step - loss: 163.5903 - val_loss: 71.2654
Epoch 9/200
34/34 [==============================] - 2s 51ms/step - loss: 167.0464 - val_loss: 71.0082
Epoch 10/200
34/34 [===========================

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testImagesX)

# compute the difference between the *predicted* house prices and the *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 64.40%, std: 70.58%


Here it is the example that combines numerical data and images: https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/.